<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Ciencia_de_Dados/blob/main/Spark_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master('local[*]')\
    .appName("Regressao com Spark")\
    .getOrCreate()

spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dados = spark.read.json('/content/drive/MyDrive/Arquivos/imoveis.json')

In [ ]:
dados

In [ ]:
dados.show(truncate=False)

In [ ]:
dados.count()

In [ ]:
dados.printSchema()

In [ ]:
from os import truncate
dados\
    .select('ident.customerID', 'listing.*') \
    .show(truncate=False)

In [ ]:
dados\
    .select("ident.customerID", "listing.types.*", "listing.features.*", "listing.address.*", "listing.prices.price", "listing.prices.tax.*")\
    .show(truncate=False)

In [ ]:
dados\
    .select("ident.customerID", "listing.types.*", "listing.features.*", "listing.address.*", "listing.prices.price", "listing.prices.tax.*")\
    .drop("city", "location", "totalAreas")\
    .show(truncate=False)


In [ ]:
dataset = dados\
    .select("ident.customerID", "listing.types.*", "listing.features.*", "listing.address.*", "listing.prices.price", "listing.prices.tax.*")\
    .drop("city", "location", "totalAreas")\
    #.show(truncate=False)


In [ ]:
dataset.printSchema()


In [ ]:
from pyspark.sql.types import IntegerType, DoubleType

In [ ]:
dataset\
    .withColumn('usableAreas', dataset['usableAreas'].cast(IntegerType()))\
    .withColumn('price', dataset['price'].cast(DoubleType()))\
    .withColumn('condo', dataset['condo'].cast(DoubleType()))\
    .withColumn('iptu', dataset['iptu'].cast(DoubleType()))\
    .printSchema()

In [ ]:
dataset = dataset\
    .withColumn('usableAreas', dataset['usableAreas'].cast(IntegerType()))\
    .withColumn('price', dataset['price'].cast(DoubleType()))\
    .withColumn('condo', dataset['condo'].cast(DoubleType()))\
    .withColumn('iptu', dataset['iptu'].cast(DoubleType()))

In [ ]:
dataset.show()

In [ ]:
dataset\
    .select('usage')\
    .groupBy('usage')\
    .count()\
    .show()

In [ ]:
dataset\
    .select('*')\
    .where('usage=="Residencial"')

In [ ]:
dataset\
    .select('unit')\
    .groupBy('unit')\
    .count()\
    .show()

In [ ]:
dataset\
    .select('zone')\
    .groupBy('zone')\
    .count()\
    .show()

In [ ]:
from pyspark.sql import functions as f

In [ ]:
dataset\
    .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dataset.columns])\
    .show()

In [ ]:
dataset\
    .select('*')\
    .na\
    .fill(0)\
    .show()

In [ ]:
dataset = dataset\
    .select('*')\
    .na\
    .fill(0)

In [ ]:
dataset\
    .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dataset.columns])\
    .show()

In [ ]:
dataset\
    .select('zone')\
    .groupBy('zone')\
    .count()\
    .show()


In [ ]:
dataset = dataset\
    .where(f.col('zone') != '')

In [ ]:
dataset.show()

###Preparando os Dados


##Variáveis Dummy

In [ ]:
dataset\
    .groupBy('customerID')\
    .pivot('unit')\
    .agg(f.lit(1))\
    .na\
    .fill(0)\
    .show()

In [ ]:
unit = dataset\
    .groupBy('customerID')\
    .pivot('unit')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

In [ ]:
dataset\
    .groupBy('customerID')\
    .pivot('zone')\
    .agg(f.lit(1))\
    .na\
    .fill(0)\
    .show()

In [ ]:
zone = dataset\
    .groupBy('customerID')\
    .pivot('zone')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

In [ ]:
dataset = dataset\
    .join(unit, 'customerID', how='inner')\
    .join(zone, 'customerID', how='inner')

In [ ]:
dataset.show()

##Vetoriacao

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
dataset = dataset.withColumnRenamed('price','label')

In [ ]:
X = [
    'bathrooms',
    'bedrooms',
    'floors',
    'parkingSpaces',
    'suites',
    'unitFloor',
    'unitsOnTheFloor',
    'usableAreas',
    'condo',
    'iptu',
    'Apartamento',
    'Casa',
    'Outros',
    'Zona Central',
    'Zona Norte',
    'Zona Oeste',
    'Zona Sul'
]

In [ ]:
assembler = VectorAssembler(inputCols=X, outputCol= 'features')
assembler

In [ ]:
dataset.show()

In [ ]:
dataset_prep = assembler.transform(dataset).select('features', 'label')

In [ ]:
dataset_prep.show(10, truncate=False)

In [ ]:
from pyspark.ml.stat import Correlation
import pandas as pd


In [ ]:
correlacao = Correlation.corr(dataset_prep, 'features').collect()

In [ ]:
correlacao

In [ ]:
correlacao = Correlation.corr(dataset_prep, 'features').collect()[0][0]

In [ ]:
correlacao

In [ ]:
correlacao.toArray()

In [ ]:
dataframe_correlacao = pd.DataFrame(correlacao.toArray(), columns=X, index=X)

In [ ]:
dataframe_correlacao

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,10))
paleta = sns.color_palette("light:salmon", as_cmap=True)
sns.heatmap(dataframe_correlacao.round(1), annot=True, cmap=paleta)

## Ajuste e Previsão

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
treino, teste = dataset_prep.randomSplit([0.7, 0.3], seed=101)

In [ ]:
treino.count()

In [ ]:
teste.count()

In [ ]:
lr = LinearRegression()

In [ ]:
modelo_lr = lr.fit(treino)

In [ ]:
previsoes_lr_treino = modelo_lr.transform(treino)

In [ ]:
previsoes_lr_treino.show()

##Métricas

In [ ]:
resumo_treino = modelo_lr.summary

In [ ]:
resumo_treino.r2

In [ ]:
resumo_treino.rootMeanSquaredError

In [ ]:
resumo_teste = modelo_lr.evaluate(teste)

In [ ]:
resumo_teste.r2

In [ ]:
resumo_teste.rootMeanSquaredError

###Tabela Resumo Regressão Linear

In [ ]:
print('Linear Regression')
print("")
print("="*30)
print("Dados de Treino")
print("="*30)
print("R²: %f" % resumo_treino.r2)
print("RMSE: %f" % resumo_treino.rootMeanSquaredError)
print("")
print("="*30)
print("Dados de Teste")
print("="*30)
print("R²: %f" % resumo_teste.r2)
print("RMSE: %f" % resumo_teste.rootMeanSquaredError)


##Aula - 3 Decision Tree

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
dtr = DecisionTreeRegressor(seed=101, maxDepth=7)

In [ ]:
modelo_dtr = dtr.fit(treino)

In [ ]:
previsoes_dtr_treino = modelo_dtr.transform(treino)

In [ ]:
previsoes_dtr_treino.show()

##Métricas

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator


In [ ]:
evaluator = RegressionEvaluator()

print(evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: "r2"}))
print(evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: "rmse"}))


In [ ]:
previsoes_dtr_teste = modelo_dtr.transform(teste)

In [ ]:
previsoes_dtr_teste.show()

In [ ]:
print('Decision Tree Regression')
print("="*30)
print("Dados de Treino")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: "rmse"}))
print("")
print("="*30)
print("Dados de Teste")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: "rmse"}))


## Randon Forest Refgressor

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(seed=101, maxDepth=7, numTrees=10)

In [ ]:
modelo_rfr = rfr.fit(treino)

In [ ]:
previsoes_rfr_treino = modelo_rfr.transform(treino)

In [ ]:
previsoes_rfr_treino.show()

###Métricas

In [ ]:
print(evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: "r2"}))
print(evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: "rmse"}))

In [ ]:
previsoes_rfr_teste = modelo_rfr.transform(teste)

In [ ]:
previsoes_rfr_teste.show()

In [ ]:
print('Random Forest Regression')
print("="*30)
print("Dados de Treino")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: "rmse"}))
print("")
print("="*30)
print("Dados de Teste")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_rfr_teste, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_rfr_teste, {evaluator.metricName: "rmse"}))


###Árvore de Decisão com Cross Validation

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
dtr = DecisionTreeRegressor()

In [ ]:
grid = ParamGridBuilder() \
    .addGrid(dtr.maxDepth, [2,5,10]) \
    .addGrid(dtr.maxBins, [10, 32, 45]) \
    .build()

In [ ]:
evaluator = RegressionEvaluator()

In [ ]:
dtr_cv = CrossValidator(
    estimator=dtr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed = 101

)

In [ ]:
modelo_dtr_cv = dtr_cv.fit(treino)

In [ ]:
previsoes_dtr_cv_teste = modelo_dtr_cv.transform(teste)

In [ ]:
print('Decision Tree Regression')
print("="*30)
print("Sem Cross Validation")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: "rmse"}))
print("")
print("="*30)
print("Com Cross Validation")
print("="*30)
print("R²: %f" % evaluator.evaluate(previsoes_dtr_cv_teste, {evaluator.metricName: "r2"}))
print("RMSE: %f" % evaluator.evaluate(previsoes_dtr_cv_teste, {evaluator.metricName: "rmse"}))
